In [ ]:
def yolov8_predict(
    model_path = 'last_288.pt',
    src_path = 'sample_total_41893',
    predict_path = 'predict_288epochs',
    predict_txt_path = 'predict.txt',
    THRESHOLD = 0.2,
    mapping_path = 'mapping.txt',
    is_mapping_from_txt = True,
    mapping_reverse = True, 
    category_length = 31,
    is_save = True, #예측 save
    only_category = False, #해당 카테고리만 저장
    ):
    from collections import defaultdict
    from ultralytics import YOLO
    import os
    import cv2
    import numpy as np
    from PIL import Image 
    
    #카테고리 우선순위 
    # def return_sorted_category(category_map = {1:30,2:0,3:4,4:10,5:16},
    #                         compare_list = [16, 4, 0, 30,10],
    #                         ):
        
    #     # Reverse the map_dict for easier look-up
    #     reverse_map = {v: k for k, v in category_map.items()}

    #     # Sort compare_list based on their corresponding keys in map_dict
    #     sorted_list = sorted(compare_list, key=lambda x: reverse_map.get(x, float('inf')))

    #     print(sorted_list)
    #     return sorted_list
    #     #[30, 0, 4, 10, 16]
    
    # Function to draw bounding boxes on an image
    def draw_boxes(image, boxes, names, conf):
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            class_id = int(conf[i])
            class_name = names[class_id]
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Draw rectangle
            cv2.putText(image, class_name, (x1 + 5, y1 + 20)# - 10)
                        , cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)  # Draw text
            #break 
    
    image_list = os.listdir(src_path)
    # Initialize match_count_dict
    match_count_dict = defaultdict(lambda: defaultdict(int))
    model = YOLO(model_path)
    mapping = {}
    
    test_list = []
    
    if is_mapping_from_txt:
        #소분류 392
        with open(mapping_path,'r') as f:
            lines = f.readlines()
        
        for line in lines:
                # Split the line into a key and a value
            value, key = line.strip().split(',')
            
            if mapping_reverse:
                mapping[value] = key
            else:

                # Insert the key and value into the dictionary
                mapping[key] = value  # convert the key to int because your keys are integers
                
    
    #print(mapping)
    #return 
        
    
    test_count = 0
    for image in image_list:
        
        #텍스트파일 스킵 
        if '.txt' in image:
            continue
        
        #카테고리 체크 
        if image[:6] not in mapping:
            continue
    
             
        CATEGORY_ID = mapping[image[:6]]
        #if CATEGORY_ID != '11':
            #continue
   
        image_path = os.path.join(src_path,image)
        encoded_path = cv2.imdecode(np.fromfile(image_path,dtype=np.uint8),cv2.IMREAD_UNCHANGED)
        # Run inference on 'bus.jpg' with arguments
        result = model.predict(encoded_path,conf=THRESHOLD) #, save=True)#, imgsz=320, conf=0.2,classes=[0,1,2,3])

        for ret in result: 
            boxes = ret.boxes
            names = ret.names  
            break

        # Extracting class IDs and confidence scores
        cls = boxes.cls.cpu().numpy() # class IDs
        conf = boxes.conf.cpu().numpy() # confidence scores
        
        if is_save:
            if not os.path.exists(os.path.join(predict_path,CATEGORY_ID)):
                os.makedirs(os.path.join(predict_path,CATEGORY_ID)) 
         
            #predict 카테고리 folder 생성
            if not os.path.exists(os.path.join(predict_path,CATEGORY_ID,image[:6])):
                os.makedirs(os.path.join(predict_path,CATEGORY_ID,image[:6]))
                
            # Draw the bounding boxes on the image
            draw_boxes(encoded_path, boxes.xyxy.cpu().numpy(), names, cls)
            # Save the image
            save_path = os.path.join(os.path.join(predict_path,CATEGORY_ID,image[:6]), image.encode('utf-8').decode('utf-8'))
            
            if only_category == False: 
                Image.fromarray(cv2.cvtColor(encoded_path, cv2.COLOR_BGR2RGB)).save(save_path)
        
        #print('cls = ',cls)
        #print('conf = ',conf)
        # Iterating through the results
        
        #cls 인식못하는것도 있슴 predict 할 때 threshold 에 의한 
        #cls list 는 내림차순 descendent 로 정렬 
        if len(cls) == 0:
            test_count += 1 
            #print('cls 인식못하는것도 있슴 = ',)
            #break 
            
        #카테고리별 우선순위 
        # test_cls = [] 
        # same_category = False
        # if len(cls) > 1: #2개이상 잡혓을때
        #     for i in range(len(cls)):
        #         class_id = int(cls[i])
        #         confidence = conf[i]
        #         class_name = names[class_id]
        #         if confidence >= THRESHOLD:
        #             test_cls.append(class_id)
                   
        #             #같은 카테고리 우선권 
        #             if class_id == int(CATEGORY_ID):
        #                 match_count_dict[image[:6]][class_id] += 1 
        #                 same_category = True
        #                 break 
            
        #     #같은 카테고리 없을시 threshold 높은걸로            
        #     if same_category == False:
        #         match_count_dict[image[:6]][test_cls[0]] += 1 
        #         #print('test_cls = ',test_cls,'category = ',CATEGORY_ID,'vienna = ',image[:6])

        #     # if test_cls.__len__() >= 3:
        #     #     print(test_cls)
        #     #     print(conf)
        #     #     break

        # else:
        #     for i in range(len(cls)):
        #         class_id = int(cls[i])
        #         confidence = conf[i]
        #         class_name = names[class_id]
        #         if confidence >= THRESHOLD:
        #              match_count_dict[image[:6]][class_id] += 1
                
        #         break 
        # continue     
        
        for i in range(len(cls)):
            class_id = int(cls[i])
            confidence = conf[i]
            class_name = names[class_id]
            #test_box = boxes.xyxy.cpu().numpy()
            #x1, y1, x2, y2 = map(int, test_box[i])
            if confidence >= THRESHOLD:
                #print('above confidence > ', confidence, ' class_id = ',class_id)
                #if class_id == int(CATEGORY_ID):
                match_count_dict[image[:6]][class_id] += 1
                
                if only_category:
                    if class_id == int(CATEGORY_ID):
                        Image.fromarray(cv2.cvtColor(encoded_path, cv2.COLOR_BGR2RGB)).save(save_path)
                #match_count_dict[int(CATEGORY_ID)][class_id] += 1 #= match_count_dict.get(CATEGORY_ID, 0) + 1 
                    #print(f"Class ID: {class_id}, Class Name: {class_name}, Confidence: {confidence * 100:.2f}%")
                #else:
                    #pass
                    #print(f'below confidence : class_id = {class_id} confidence = {confidence} class_name = {class_name}\n')
                    #test_list.append(f'below confidence : class_id = {class_id} confidence = {confidence} class_name = {class_name}\n')
            else:
                #print('below confidence < ',confidence)
                pass
                #print(f'below confidence : class_id = {class_id} confidence = {confidence} class_name = {class_name}\n')
                #test_list.append(f'below confidence : class_id = {class_id} confidence = {confidence} class_name = {class_name}\n')
            break 
        
    
    with open(predict_txt_path,'w') as file:
        #for key in match_count_dict.keys():
            #file.write(f'{key} = {match_count_dict[key]}\n')   
        #print('txt done!!') 
        #print('mapping length = ',len(mapping))
        for key,inner_dict in match_count_dict.items():
            line = f'{key}= ' #{inner_dict[key]} '
            for i in range(category_length):
                #if i != int(key):
                line += f'{inner_dict.get(i,0)} '
            file.write(line.strip() + '\n')

yolov8_predict() 